In [1]:
SEED = 12
BATCH_SIZE = 64
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from sklearn.model_selection import train_test_split

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device} device")

Using cuda device


In [3]:
class CCFDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.labels = self.data.iloc[:, -1]
        self.features = self.data.iloc[:, :-1]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32).to()
        label = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return features, label

# Example usage:
dataset = CCFDataset("/kaggle/input/creditcardfraud/creditcard.csv")
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=SEED)



train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, pin_memory = True if device == "cuda" else False)
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, pin_memory = True if device == "cuda" else False)
for X, y in test_dl:
    print(f"Shape of X [Time, V1-28, Amount]: {X.shape}")
    print(f"Shape of y [Class]: {y.shape} {y.dtype}")
    break

Shape of X [Time, V1-28, Amount]: torch.Size([64, 30])
Shape of y [Class]: torch.Size([64]) torch.int64


In [4]:
class ModelNN (nn.Module):
    def __init__(self):
        super().__init__()
        self.dense_relu_stack = nn.Sequential(
            nn.Linear(30,64),
            nn.ReLU(),
            nn.Linear(64,64),
            nn.ReLU(),
            nn.Linear(64,1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        return self.dense_relu_stack(x)
model = ModelNN().to(device)
print(model)        

ModelNN(
  (dense_relu_stack): Sequential(
    (0): Linear(in_features=30, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [5]:
##TODO
##Training Loop
##Optimization
##Explore other techniques